In [1]:
%%capture
!pip install opencv-python
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
# Function to convert video to frames
def video_to_frames(video_path, frames_folder,m=30):
    # Create a VideoCapture object
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Read until video is completed
    frame_count = 0
    kept_count=0
    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret:
            # Save frame as JPEG file
            frame_count += 1
            if frame_count%m==0:
                frame_path = Path(frames_folder) / f"frame_{frame_count}.jpg"
                kept_count+=1
                cv2.imwrite(str(frame_path), frame)
        else:
            break
    
    # When everything done, release the video capture object
    cap.release()
    print(f"Video has been split into {frame_count} frames.")
    print(f"number of frame we will use {kept_count} frames.")

In [3]:
# Function to discard unclear frames
def discard_unclear_frames(frames_folder, clarity_threshold):
    frames_path = Path(frames_folder)
    clear_frames = []
    
    for frame_path in frames_path.glob("*.jpg"):
        frame = cv2.imread(str(frame_path))
        # Calculate the Laplacian variance (focus measure)
        variance = cv2.Laplacian(frame, cv2.CV_64F).var()
        if variance > clarity_threshold:
            clear_frames.append(frame)
        else:
            frame_path.unlink()  # Delete unclear frame
    
    print(f"{len(clear_frames)} clear frames have been kept for stitching.")

In [4]:
# Function to stitch frames together
def stitch_frames(frames, output_path):
    # Create a stitcher object
    stitcher = cv2.Stitcher_create()
    
    # Perform the stitching process
    status, stitched_image = stitcher.stitch(frames)
    
    # Check if the stitching is successful
    if status == cv2.Stitcher_OK:
        # Save the stitched image
        cv2.imwrite(output_path, stitched_image)
        print("Frames have been successfully stitched together.")
        stitched_image = cv2.imread(output_path)
        plt.imshow(cv2.cvtColor(stitched_image, cv2.COLOR_BGR2RGB))
        plt.title('Stitched Image')
        plt.axis('off')
        plt.show()
    else:
        print("Frames could not be stitched together. Error code: " + str(status))
    

In [5]:
def clear_frames_directory(frames_folder):
    # Get the folder path
    folder_path = Path(frames_folder)
    
    # Check if the folder exists
    if folder_path.exists() and folder_path.is_dir():
        # Delete all files in the folder except .ipynb_checkpoints
        for file_path in folder_path.glob('*'):
            if not file_path.name.startswith('.ipynb_checkpoints'):
                file_path.unlink()
    else:
        print("Error: Frames folder does not exist.")

In [6]:
video_path = 'bss/videos/bottle2.mp4'
frames_folder = 'bss/images'
clarity_threshold = 100  # Example threshold value for clarity

clear_frames_directory(frames_folder)

# Convert video to frames
video_to_frames(video_path, frames_folder,m=5)

# Discard unclear frames
discard_unclear_frames(frames_folder, clarity_threshold)

# Load clear frames
clear_frames = [cv2.imread(str(frame_path)) for frame_path in Path(frames_folder).glob("*.jpg")]

# Stitch frames together
stitch_frames(clear_frames, 'bss/images/stitched_image.jpg')

Error: Frames folder does not exist.
Error: Could not open video.
0 clear frames have been kept for stitching.
Frames could not be stitched together. Error code: 1


In [21]:
import cv2
import numpy as np

def detect_keypoints(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Gray', gray)
    
    # Step 1: Apply Gaussian Blur
    blurred = cv2.GaussianBlur(gray, (19, 19), 0)

    cv2.imshow('Gray', blurred)
    # Step 2: Canny Edge Detection
    edges = cv2.Canny(blurred, 50, 150)

    cv2.imshow('Gray', edges)
    
    # Step 3: Hough Line Transform to find vertical lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Assuming the image might need rotation to make lines vertical
    # Rotate image if necessary (not implemented here)

    # Step 4: Hough Transform for Ellipses (OpenCV does not directly support ellipse detection via Hough Transform, this is a simplification)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(image, (x, y), r, (255, 0, 0), 4)

    # Display the result
    cv2.imshow('Detected Key Points', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
detect_keypoints('images/Screenshot 2024-04-23 122236.png')